In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('../recordFPT - recode/base/4process_25Nov.xlsx')

In [3]:
df.head(5)

,STT,DV_Số điện thoại,DV_Số hợp đồng,DV_Dịch vụ,TT_Vùng,TT_Giới tính,TT_Nhóm tuổi,TT_Mức độ am hiểu,TT_Mục đích sử dụng,TT_Thời gian sử dụng,TT_Note,KL_Nội dung hỗ trợ,KL_flow,KL_Issue,KL_Success,KL_Warning,KL_Consideration,KL_Đánh giá NV + đề xuất
0,1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,31.0,-,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5\nn34=s1>hd5,NaN,NaN,NaN,NaN,NaN
1,2,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,31.0,KH say ke,Thanh lý hợp đồng internet,n00=s1>ds1+dx4\nn10=s1>ds3\s2>ds1\s3>ll3+dx5\n...,NaN,NaN,NaN,NaN,NaN
2,3,0973641746,HNAAG3366,Giga,Hà Nội,Nam,35 - 65,Có hiểu biết,-,30.0,-,Tạm ngưng hợp đồng internet,n00=s1>gt1+ds1\nn22=s1>ll3+dx4\s2>dx5\nn34=s1>...,NaN,NaN,NaN,NaN,NaN
3,4,0912277171,HNAAJ5017,Giga,Hà Nội,Nam,35 - 65,Có hiểu biết,-,29.0,-,Thanh lý hợp đồng internet,n16=s1>ds4+dx5\s2>ll4+dx5\nn34=s1>hd1+hd5,NaN,NaN,NaN,NaN,NaN
4,5,0916206466,HNJ093207,Giga,Hà Nội,Nam,18 - 35,Có hiểu biết,-,36.0,-,Thanh lý hợp đồng internet,n00=s1>ds1\s2>ds3+dx4\nn12=s1>ds4\nn11=s1>dx7\...,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df.dropna(subset=['KL_flow'])
df['KL_flow'].isna().sum()
df.drop(columns=['STT','DV_Số điện thoại', 'DV_Số hợp đồng', 'DV_Dịch vụ',
       'TT_Vùng', 'TT_Giới tính', 'TT_Nhóm tuổi', 'TT_Mức độ am hiểu',
       'TT_Mục đích sử dụng', 'TT_Thời gian sử dụng', 'TT_Note','KL_Issue', 'KL_Success', 'KL_Warning',
       'KL_Consideration', 'KL_Đánh giá NV + đề xuất'],
        inplace=True)

In [5]:
def split_and_index_kl_flow(df):
    
    df['KL_flow'] = df['KL_flow'].astype(str)

    # Create a new dataframe to store the processed data
    processed_data = []
    
    # Iterate through each row in the original dataframe
    for idx, row in df.iterrows():
        # stt = row['STT']
        kl_flow = row['KL_flow']
        
        # Split the KL_flow into multiple lines based on '\n'
        flow_lines = kl_flow.split('\n')
        
        # Iterate through each flow line and create sub_ids
        for sub_id, flow in enumerate(flow_lines, start=1):
            new_row = row.copy()  # Copy the row to keep other columns the same
            # new_row['sub_id'] = f"{stt}.{sub_id}"  # Create sub_id by combining STT and sub_id
            new_row['KL_flow'] = flow  # Assign the current flow line to KL_flow
            processed_data.append(new_row)
    
    # Convert the processed data into a new DataFrame
    processed_df = pd.DataFrame(processed_data)
    
    # Reorder columns: Insert sub_id right after STT
    cols = list(processed_df.columns)
    # cols.insert(1, cols.pop(cols.index('sub_id')))  # Move sub_id to be the second column
    processed_df = processed_df[cols]
    
    # Reset the index
    processed_df.reset_index(drop=True, inplace=True)
    
    return processed_df

In [6]:
df = split_and_index_kl_flow(df)

In [7]:
def extract_issue(df):
    # Create a list to store split_flow values
    split_flow_list = []
    
    for index, row in df.iterrows():
        flow_value = row['KL_flow']
        
        # Check if the value is a string and not NaN
        if isinstance(flow_value, str):
            split_flow = flow_value.split("=", 1)[0]
            
            if "#" in split_flow:
                split_flow = split_flow.replace("#", "")

            if "[" in split_flow:
                split_flow = split_flow.split("[", 1)[0]
                
            split_flow_list.append(split_flow)  # Append to the list
        else:
            split_flow_list.append(None)  # Append None for invalid values
            print(f"Row {index}: Invalid Flow value (not a string): {flow_value}")

    # Add the new column to the DataFrame
    df['Reason'] = split_flow_list

    return df

In [8]:
df = extract_issue(df)

In [9]:
df

,KL_Nội dung hỗ trợ,KL_flow,Reason
0,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5,n12
1,Thanh lý hợp đồng internet,n34=s1>hd5,n34
2,Thanh lý hợp đồng internet,n00=s1>ds1+dx4,n00
3,Thanh lý hợp đồng internet,n10=s1>ds3\s2>ds1\s3>ll3+dx5,n10
4,Thanh lý hợp đồng internet,n34=s1>hd1+hd5,n34
...,...,...,...
1234,Thanh lý hợp đồng truyền hình trong combo,n21[truyền hình]=s1'>ll2+ds2,n21
1235,Thanh lý hợp đồng truyền hình trong combo,n20[internet]=s1'>gt3+dx9[combo giga cộng tru...,n20
1236,Thanh lý hợp đồng truyền hình trong combo,n21#[truyền hình]=s1!>ll1+ll2+ll16+gt1,n21
1237,Thanh lý hợp đồng truyền hình trong combo,n50=s1>ds1\s2>ll3+ll5>s3>dx10[6+1 hoặc 12+2],n50


In [10]:
def extract_result(df):

    updated_df = df.copy()
    
 
    updated_df['Result'] = 'Không thành công'
    
    # Duyệt qua các dòng và xác định kết quả với thứ tự ưu tiên ' > ^ > !
    for index, row in updated_df.iterrows():
        if "'" in row['KL_flow']:
            updated_df.at[index, 'Result'] = "Thành công"
        elif '^' in row['KL_flow']:
            updated_df.at[index, 'Result'] = "Cân nhắc"
        elif '!' in row['KL_flow']:
            updated_df.at[index, 'Result'] = "Lưu ý"
    
    return updated_df


In [11]:
df = extract_result(df)

In [12]:
df.drop(columns=["KL_flow"], inplace=True)

In [13]:
def calculate_reason_statistics(df):
    # Đếm số lần xuất hiện của mỗi "Reason"
    reason_counts = df['Reason'].value_counts().reset_index()
    reason_counts.columns = ['Reason', 'Số lần xuất hiện']

    # Tính tỷ lệ phần trăm và tỷ lệ tích lũy
    reason_counts['Tỷ lệ phần trăm'] = (reason_counts['Số lần xuất hiện'] / reason_counts['Số lần xuất hiện'].sum()) * 100
    reason_counts['Tỷ lệ tích lũy'] = reason_counts['Tỷ lệ phần trăm'].cumsum()

    # Đếm số lần xuất hiện của từng Result cho mỗi Reason
    result_counts = df.groupby(['Reason', 'Result']).size().unstack(fill_value=0)

    # Tính tỷ lệ thành công và tỷ lệ cân nhắc
    result_counts['Tỷ lệ thành công'] = (result_counts['Thành công'] / result_counts.sum(axis=1)) * 100
    result_counts['Tỷ lệ cân nhắc'] = (result_counts['Cân nhắc'] / result_counts.sum(axis=1)) * 100

    # Thêm các cột tỷ lệ thành công và tỷ lệ cân nhắc vào bảng reason_counts
    reason_counts = reason_counts.set_index('Reason')
    reason_counts = reason_counts.join(result_counts[['Tỷ lệ thành công', 'Tỷ lệ cân nhắc']], on='Reason')

    # Reset lại chỉ mục cho dễ nhìn
    reason_counts = reason_counts.reset_index()

    return reason_counts

In [14]:
pareto_net = calculate_reason_statistics(df[df['KL_Nội dung hỗ trợ'] == 'Thanh lý hợp đồng internet'])

pareto_tvcombo = calculate_reason_statistics(df[df['KL_Nội dung hỗ trợ'] == 'Thanh lý hợp đồng truyền hình trong combo'])

In [18]:
pareto_net

,Reason,Số lần xuất hiện,Tỷ lệ phần trăm,Tỷ lệ tích lũy,Tỷ lệ thành công,Tỷ lệ cân nhắc
0,n00,135,20.209581,20.209581,6.666667,50.823529
1,n12,54,8.083832,28.293413,18.518519,28.958121
2,n01,49,7.335329,35.628743,10.204082,35.470527
3,n40,41,6.137725,41.766467,46.341463,4.579726
4,n16,34,5.089820,46.856287,11.764706,26.221080
5,n15,27,4.041916,50.898204,22.222222,22.347630
6,n20,26,3.892216,54.790419,38.461538,7.756563
7,n10,23,3.443114,58.233533,21.739130,17.881438
8,n11,22,3.293413,61.526946,9.090909,6.432749
9,n14,21,3.143713,64.670659,14.285714,8.502024


In [15]:
pareto_tvcombo

,Reason,Số lần xuất hiện,Tỷ lệ phần trăm,Tỷ lệ tích lũy,Tỷ lệ thành công,Tỷ lệ cân nhắc
0,n21,137,40.412979,40.412979,56.204380,11.904492
1,n00,64,18.879056,59.292035,7.812500,52.915579
2,n25,18,5.309735,64.601770,55.555556,2.719033
3,n20,18,5.309735,69.911504,61.111111,1.264045
4,n40,15,4.424779,74.336283,53.333333,1.463415
5,n35,14,4.129794,78.466077,28.571429,2.348993
6,n01,9,2.654867,81.120944,22.222222,19.217082
7,n27,8,2.359882,83.480826,75.000000,1.204819
8,n60,8,2.359882,85.840708,50.000000,3.448276
9,n36,7,2.064897,87.905605,71.428571,0.000000


In [17]:
output_file = 'pareto.xlsx'

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    pareto_net.to_excel(writer, sheet_name='Thanh lý hợp đồng internet', index=False)
    pareto_tvcombo.to_excel(writer, sheet_name='Thanh lý hợp đồng tv combo', index=False)